# Project Week 2

Emiel Steegh   - s1846388  
Freek Nijweide - s1857746

# Project Description

Description etc etc

## Assignment 1

text

## Assignment 2

text

## Assignment 3

In [1]:
# code

## Assignment 4

In [2]:
# code